# LSTM, GRU를 이용한 시계열 발전량 예측 모델

## Preprocess

In [1]:
import pandas as pd
import numpy as np
import os
import random
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

df_train = pd.read_csv('train/train.csv')
submission = pd.read_csv('sample_submission.csv')

dfs_test = []

for i in range(81):
    temp_df = pd.read_csv(f"test/{i}.csv")
    dfs_test.append(temp_df.copy())

In [3]:
import math
    
def Make_New_Features(df_train):
    df_train['GHI'] = df_train['DHI'] + df_train['DNI']
    df_train['RDHNI'] = df_train['GHI'].apply(lambda x : x**0.5)
#====================================================================================
    df_train['Lower_Cloud'] = 2.5 * df_train['RH'] / 100 - 1.5
    df_train['Middle_Cloud'] = 4 * df_train['RH'] / 100 - 3.0
#====================================================================================
    b = 17.62
    c = 243.12
    df_train['gamma'] = (b * df_train['T'] /(c + df_train['T'])) + (df_train['RH'] / 100.0).apply(lambda x: math.log(x + 1))
    df_train['DD'] = (c * df_train['gamma']) / (b - df_train['gamma'])

    df_train.drop('gamma', axis=1, inplace=True)
#====================================================================================
        
    return df_train

In [4]:
from sklearn import preprocessing

scalers ={}

def preprocess(df, make_scaler=True):
    df = df.copy()
    df = Make_New_Features(df)
    
    df['Minute'].replace(30, 0.5, inplace=True)
    df['Time'] = df['Hour'] + df['Minute']
    
    df['Time_x'] = np.cos((df['Time'] * 2 * np.pi) / 24.0)
    df['Time_y'] = np.sin((df['Time'] * 2 * np.pi) / 24.0)
    
    scale_target_columns = ['TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'GHI', 'RDHNI', 'Lower_Cloud',
                           'Middle_Cloud', 'DD']
    
    temp_df_agg = pd.DataFrame()
    for c in scale_target_columns:
        temp_df_agg[f'max_week_{c}'] = df.groupby('Day')[c].max().rolling(7, center=True).max()
        temp_df_agg[f'min_week_{c}'] = df.groupby('Day')[c].min().rolling(7, center=True).min()

        temp_df_agg[f'max_week_{c}'].iloc[:3] = temp_df_agg.iloc[3][f'max_week_{c}']
        temp_df_agg[f'max_week_{c}'].iloc[-3:] = temp_df_agg.iloc[-4][f'max_week_{c}']
        temp_df_agg[f'min_week_{c}'].iloc[:3] = temp_df_agg.iloc[3][f'min_week_{c}']
        temp_df_agg[f'min_week_{c}'].iloc[-3:] = temp_df_agg.iloc[-4][f'min_week_{c}']
        
        
    temp_df_agg = temp_df_agg.reset_index()
    df = pd.merge(df, temp_df_agg, left_on='Day', right_on='Day')
    for c in scale_target_columns:
        df[f'scaled_{c}'] = (df[c] - df[f'min_week_{c}'] + 1) / (df[f'max_week_{c}']+1)

    df['peak_diff'] = df['max_week_TARGET'] - df['TARGET']
    df['peak_diff_scaled'] = df['peak_diff'] / df['max_week_TARGET']
    
    
    normalize_target_column = ['DHI', 'DNI', 'WS', 'RH', 'T', 'GHI', 'RDHNI', 'TARGET',
                               'Lower_Cloud', 'Middle_Cloud', 'DD']
    
    for c in normalize_target_column:
        x = df[[c]].copy() #returns a numpy array
        if make_scaler:
            std_scaler = preprocessing.MinMaxScaler()
            std_scaler.fit(x)
            df[c] = std_scaler.transform(x)
            scalers[c] = std_scaler
        else:
            df[c] = scalers[c].transform(x)
    
    df = df.drop(['Hour', 'Minute', 'Time'], axis=1)
    
    return df

df = preprocess(df_train)

In [5]:
temp = []
for df_test in dfs_test:
    temp.append(preprocess(df_test, make_scaler=False))
dfs = temp

In [6]:
def make_train_data_set(df):
    df = df.astype('float')
    ins_datas = df[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'GHI', 'RDHNI', 'Lower_Cloud',
                   'Middle_Cloud', 'DD', 'Time_x', 'Time_y',
                   'scaled_TARGET', 'peak_diff_scaled']].values
    outs_datas = df[['TARGET']].values
    
    ins = []
    outs = []
    
    for i in range(len(df)-48*9):
        ins.append(ins_datas[i:i+48*7])
        outs.append(outs_datas[i+48*7:i+48*8])
        
    return np.array(ins), np.array(outs)

def make_prediction_data_set(df):
    df = df.astype('float')
    ins_datas = df[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'GHI', 'RDHNI', 'Lower_Cloud',
                   'Middle_Cloud', 'DD', 'Time_x', 'Time_y',
                   'scaled_TARGET', 'peak_diff_scaled']].values
    
    ins = [ins_datas]

    return np.array(ins)

ins, outs = make_train_data_set(df)
ins.shape, outs.shape

((52128, 336, 15), (52128, 48, 1))

In [7]:
term = 17376

x_year_1 = ins[:term]
x_year_2 = ins[term:term*2]
x_year_3 = ins[term*2:]

y_year_1 = outs[:term]
y_year_2 = outs[term:term*2]
y_year_3 = outs[term*2:]

drop_columns=[]
x_year_1.shape, x_year_2.shape, x_year_3.shape, y_year_1.shape, y_year_2.shape, y_year_3.shape

((17376, 336, 15),
 (17376, 336, 15),
 (17376, 336, 15),
 (17376, 48, 1),
 (17376, 48, 1),
 (17376, 48, 1))

In [8]:
temp = []
for df in dfs:
    temp.append(make_prediction_data_set(df))
pre_ins = np.concatenate(temp)
pre_ins.shape

(81, 336, 15)

## Model Preparation

In [9]:
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, GRU, Conv1D
from keras.layers import Flatten, Reshape
from keras.layers import LSTM, Input, Lambda, Concatenate
from keras.layers import RepeatVector, MaxPooling1D
from keras.layers import TimeDistributed, Bidirectional
from keras import Model
from keras import backend as K
import tensorflow as tf
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

# 7일의 데이터로 2일을 예측
# 1일 = 48개의 관측값
# 7일 = 7 * 48 = 336
# 2일 = 2 * 48 = 96
def pinball_loss(tau):
    def loss(y_true, y_pred):
        err = y_true - y_pred
        return K.mean(K.maximum(tau * err, (tau - 1) * err))
    return loss


def build_model(tau, lr=0.001, use_summary=True, use_mse=False):
    opt = optimizers.Adam(lr=lr)
    
    x_lstm = Input(shape=(336, 15))
    
    x_ins = []
    for i in range(7):
        x_ins.append(Lambda(lambda x: x[:, 48*i:48*(i+1), :])(x_lstm))
    
    x_s = []
    for x in x_ins:
        x = GRU(100, return_sequences=True)(x)
        x = GRU(80, return_sequences=False)(x)
        x_s.append(x)
    
    x = Concatenate()(x_s)
    x = Reshape(target_shape=(7, 80))(x)
    x = Bidirectional(LSTM(120, return_sequences=False))(x)
    x = Dense(48, activation='relu')(x)

    model = Model(x_lstm, x)
    
    if use_mse:
        model.compile(loss='mse', optimizer=opt)
    else:
        model.compile(loss=pinball_loss(tau), optimizer=opt)
        
    if use_summary:
        model.summary()
    
    return model

In [10]:
from tqdm import tqdm

def do_dl(q, verbose=0):
    es = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True,)
    
    model_1 = build_model(q, 0.0001, use_summary=False)
    his = model_1.fit(np.concatenate([x_year_1, x_year_2]),
                     np.concatenate([y_year_1, y_year_2]),
                     epochs=300, batch_size=2000, validation_data=(x_year_3, y_year_3),
                     verbose=verbose, callbacks=[es])
    model_1.save(f'models/ensemble/model_1_lstm_1_8_{q}_1_fined')

    model_2 = build_model(q, 0.0001, use_summary=False)
    his = model_2.fit(np.concatenate([x_year_1, x_year_3]),
                     np.concatenate([y_year_1, y_year_3]),
                     epochs=300, batch_size=2000, validation_data=(x_year_2, y_year_2),
                     verbose=verbose, callbacks=[es])
    model_2.save(f'models/ensemble/model_2_lstm_1_8_{q}_1_fined')

    model_3 = build_model(q, 0.0001, use_summary=False)
    his = model_3.fit(np.concatenate([x_year_2, x_year_3]),
                     np.concatenate([y_year_2, y_year_3]),
                     epochs=300, batch_size=2000, validation_data=(x_year_1, y_year_1),
                     verbose=verbose, callbacks=[es])
    model_3.save(f'models/ensemble/model_3_lstm_1_8_{q}_1_fined')

In [ ]:
for q in tqdm(taus[1:]):
    do_dl(q, verbose=0)

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: models/ensemble/model_1_lstm_1_8_0.2_1_fined\assets
INFO:tensorflow:Assets written to: models/ensemble/model_2_lstm_1_8_0.2_1_fined\assets
INFO:tensorflow:Assets written to: models/ensemble/model_3_lstm_1_8_0.2_1_fined\assets


 12%|██████████                                                                      | 1/8 [24:22<2:50:40, 1462.98s/it]

INFO:tensorflow:Assets written to: models/ensemble/model_1_lstm_1_8_0.3_1_fined\assets
INFO:tensorflow:Assets written to: models/ensemble/model_2_lstm_1_8_0.3_1_fined\assets
INFO:tensorflow:Assets written to: models/ensemble/model_3_lstm_1_8_0.3_1_fined\assets


 25%|████████████████████                                                            | 2/8 [58:49<2:44:24, 1644.01s/it]

In [ ]:
df_sample = pd.read_csv('sample_submission.csv')


columns = ['q_0.1', 'q_0.2', 'q_0.3', 'q_0.4', 'q_0.5', 'q_0.6', 'q_0.7', 'q_0.8', 'q_0.9']

for i in range(len(columns)):
    out = do_predict(models[i])
    df_sample[columns[i]] = out
    
df_sample.to_csv(f'result_branch_from_main_model_test_2.csv', index=False)

In [ ]:
for h in hiss:
    plt.plot(h.history['loss'])
    plt.plot(h.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()